## Clean fire_violations data

In [2]:
%run 'Load_base_data.ipynb'

In [3]:
fire_violations = pd.read_csv('RawData/Fire_Violations.csv', low_memory=False)

In [4]:
fire_violations['Lat']=fire_violations['Location'].apply(get_latitude)
fire_violations['Long']=fire_violations['Location'].apply(get_longitude)
fire_violations['value'] =1

In [5]:
# convert to geopandas file type
fire_violations['Coordinates'] = list(zip(fire_violations.Long, fire_violations.Lat))
fire_violations['Coordinates'] = fire_violations['Coordinates'].apply(Point)

In [8]:
#remove data with no lat long
fire_violations=fire_violations.dropna(subset=['Lat','Long'])

In [9]:
fire_violations_main_gdf = gpd.GeoDataFrame(fire_violations, geometry='Coordinates')

In [10]:
#spatial join gps 
fire_violations_main_parcels = gpd.sjoin(fire_violations_main_gdf, geo_loc_gdf_sub, op='intersects', how='inner')

In [11]:
fire_violations_clean = pd.DataFrame(fire_violations_main_parcels)

In [ ]:
# counting the number of different violation types per address

#fire_violation_items =fire_violations[['Address','Violation Item','Lat','Long','value']].groupby(['Address','Lat','Long','Violation Item']).agg('count')
#fire_violation_items.reset_index(inplace=True)

#converting to wide format so each item is a feature
#fire_violation_items_long=fire_violation_items.pivot(index='Address',columns='Violation Item', values='value')
#fire_violation_items_long.fillna(0, inplace=True)
#fire_violation_items_long.reset_index(inplace=True)
#fire_violation_items_long.head()

In [16]:
#code the number of violations total per parcel 
fire_violation_num =fire_violations_clean[['Violation Number','blklot']].groupby(['blklot']).agg('count')
fire_violation_num.reset_index(inplace=True)
fire_violation_num.shape

(33204, 2)

In [17]:
#code the number of corrective actions per parcel
fire_violation_corrective =fire_violations_clean[['Corrective Action','blklot','value']].groupby(['blklot','Corrective Action']).agg('count')
fire_violation_corrective.reset_index(inplace=True)
fire_violation_corrective.head()

fire_violation_corrective_long=fire_violation_corrective.pivot_table(index=['blklot'],columns='Corrective Action', values='value')
fire_violation_corrective_long.fillna(0, inplace=True)
fire_violation_corrective_long.reset_index(inplace=True)

fire_violations_main= fire_violation_num.merge(fire_violation_corrective_long, left_on=['blklot'], 
                                                      right_on=['blklot'], how='left')
#fire_violations_main.head()

In [18]:
#code the status of violations per parcel
fire_violation_status =fire_violations_clean[['Status','blklot','value']].groupby(['blklot','Status']).agg('count')
fire_violation_status.reset_index(inplace=True)
fire_violation_status.head()

fire_violation_status_long=fire_violation_status.pivot_table(index=['blklot'],columns='Status', values='value')
fire_violation_status_long.fillna(0, inplace=True)
fire_violation_status_long.reset_index(inplace=True)
fire_violation_status_long.head()

fire_violations_main= fire_violations_main.merge(fire_violation_status_long, left_on=['blklot'], 
                                                      right_on=['blklot'], how='left')

In [19]:
fire_violations_main.fillna(0, inplace=True)
fire_violations_main.head()

,blklot,Violation Number,correct - no permit,obtain permit,stop all work,abated,closed,open,referred to hearing,rescinded
0,0004002,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0005001,5,3.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
2,0007001,4,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
3,0010001,4,1.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0
4,0011007,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [20]:
R2Parcel = pd.read_csv('residential_units.csv')

In [24]:
#merge with residential data to only get data only r2 buildings
fire_violations_clean_test = R2Parcel.merge(fire_violations_main,left_on=['Block and Lot Number'], 
                                                      right_on=['blklot'], how='inner')

In [25]:
fire_violations_clean_test.shape

(775, 14)

In [26]:
#check for duplicates
print(fire_violations_clean_test.shape)
print(fire_violations_clean_test[['blklot']].drop_duplicates(['blklot']).shape)

(775, 14)
(775, 1)


In [27]:
fire_violations_clean_test.to_csv('fire_violations_clean.csv')